# N-Gram

For this solution we use the N-Gram model approach. The steps will be as follows:

1. Ingest data
2. Generate vocab, encoder, decoder
3. Generate a count dictionary of all n-grams generated from our data. This will serve as our frequency lookup table.
4. Read test data.
5. Iteratively for each character:

  * generate a bigram matching the length specified earlier. Where necessary, using padding character to maintain length.

  * Use our frequency lookup table to return count of the current bigram 

  * Divide by total no of occurences of all bigrams 

  * Normalize these probabilities and return the probability matrix generated

  * Look up generated probability of our expected character

  * calculate log of the value and subtract from overall loss of the entire model

  * remove first character of our bigram and append current character to the end of the bigram


Part of the code modified from: https://www.youtube.com/watch?v=zz1CFBS4NaY



In [1]:
def suggest_next_char(input_, ngram_counts, vocab_counts, chars, encoder, len_gram, total_ngrams, total_vocab, laplace_weight=0):

    # Consider the last ngram of sentence
    tokenized_input = [x.lower() for x in input_]
    last_gram = tokenized_input[-(len_gram):]

    # Calculating probability for each char in vocab

    char_probs = {}
    total_vocab = count_unique(vocab_counts)
    unique_ngrams = count_unique(ngram_counts)

    i = 0

    start = time.time()

    for vocab_char in chars:
        i += 1
        original_gram = test_gram = "".join([last_gram[x] for x in range(len_gram)])

        test_gram = test_gram[1:]
        c_vocab = get_ngram_val(test_gram, vocab_counts)

        test_gram += str(vocab_char)
        c_ngram = get_ngram_val(test_gram, ngram_counts)
        
        prob = (c_ngram + laplace_weight) / (c_vocab + (laplace_weight*unique_ngrams))
        char_probs[vocab_char] = prob    

    top_suggestions = list(char_probs.items())

    totals = sum(map(lambda x: x[1], top_suggestions))
    coeff = 1/totals

    top_suggestions_standardized =  [tuple([x[0], x[1] * coeff]) for x in top_suggestions]
    # print("***top_suggestions_standardized***",top_suggestions_standardized)
    # print(sum(x[1] for x in top_suggestions_standardized))

    return top_suggestions_standardized

In [21]:
from math import log2, inf
import time

def evaluate_one(corpus_test, ngram_counts, vocab_counts, chars, encoder, len_gram, total_ngrams, total_vocab, laplace_weight=0):

  # input = input.lower()

  max_history = 10 #max sentence length
  # history = ['<S>'] * len_gram
  history = corpus_test[0:len_gram]  #this will serve as our seed/prompt as we're not padding
  corpus_test = corpus_test[len_gram:]

  log_loss = 0
  count = 0

  start_time = time.time()
  print("==== Running N-gram model evaluation ====")
  print("Current time: ", start_time)

  for ind, c in enumerate(corpus_test):

    count += 1

    log_loss -= log2(predict_next_proba(c, history, ngram_counts, \
                                                  vocab_counts, chars, encoder, \
                                                  len_gram, total_ngrams, total_vocab,\
                                                  laplace_weight))

    if len(history) == max_history:
      history = history[1:]

    history + str(c)

  print("Final loss ", [log_loss/count])
  print("Total time: {0} seconds".format(time.time() - start_time))    
  print("==== Finished Running N-gram model evaluation ====")

  return [log_loss/count]


def predict_next_proba(c, history, ngram_counts, vocab_counts, chars, encoder, len_gram, total_ngrams, total_vocab, laplace_weight=0):

  # pass our x into the model and return a prediction matrix
  y_pred = suggest_next_char(history, ngram_counts, vocab_counts, chars, encoder, len_gram, total_ngrams, total_vocab, laplace_weight)

  # return the computed probability of our character
  try:
    proba = y_pred[encoder[c]][1]
  except KeyError:
    proba = 1e-5 #return extremely low probability for missing keys

  return proba

In [17]:
# corpus_test[0:200177]

def test_model(corpus_test, ngram_counts, vocab_counts, chars, encoder,len_gram,total_ngrams,total_vocab,split=1):

  corpus_test = "".join(corpus_test)
  corpus_test = corpus_test[0:int(len(corpus_test)*split)]

  det = {
      "corpus_test" : corpus_test, 
      "ngram_counts" : ngram_counts, 
      "vocab_counts" : vocab_counts, 
      "chars" : chars, 
      "encoder":encoder,
      "len_gram" : len_gram,
      "total_ngrams" : total_ngrams,
      "total_vocab" : total_vocab,
      "laplace_weight" : 1
  }

  return evaluate_one(**det)

### Ingest Data

In [13]:
from sklearn.model_selection import train_test_split
import numpy as np

def get_data(path,split):
  out = {}
  corpus = open(path).readlines()[0]
  corpus = corpus.replace("\n", " ")

  # lower_case_corpus = [w.lower() for w in corpus]
  corpus = [w.lower() for w in corpus]
  print(len(corpus))
  corpus_train, corpus_test = corpus[:int(len(corpus)*split)], corpus[int(len(corpus)*split)+1:] #split into roughly 80:20 ratio 

  # for x in range(len_gram):
  #   corpus.insert(0, '<S>')

  chars = sorted(list(set(corpus_train)))
  # vocab.append('<S>')
  encoder = dict((c,i) for i,c in enumerate(chars))
  decoder = dict((i,c) for i,c in enumerate(chars))

  return corpus_train, corpus_test, chars, encoder, decoder

In [7]:
len_gram = 3

def get_ngrams(corpus_train,  len_gram):

  ngram_counts = {} # count ngram occurences
  vocab_counts = {} # count vocab occurences, for instance if len_gram is 3, ngram = trigram, vocab = bigrams
  total_ngrams = (len(corpus_train) - len_gram) + 1
  total_vocab = (len(corpus_train) - (len_gram-1)) + 1

  # Sliding through corpus to get ngram counts
  for i in range(len(corpus_train) - (len_gram-1)):
    
      # Getting ngram
      end = i + len_gram
      new_char = corpus_train[end-1]

      ngram = "".join(corpus_train[i: end])

      # Keeping track of the ngram counts
      ngram_counts = count_ngrams(ngram, ngram_counts)

  for i in range(len(corpus_train) - (len_gram-1)):

      end = i + len_gram - 1
      vocab = "".join(corpus_train[i: end])
      # keep track of vocab counts
      vocab_counts = count_ngrams(vocab, vocab_counts)

  return ngram_counts, vocab_counts, total_ngrams, total_vocab

## Test Swahili Bible

In [24]:
bbl_train, bbl_test, bbl_chars, bbl_enc, bbl_dec = get_data("/content/bible_text_.txt", 0.8)
q_train, q_test, q_chars, q_enc, q_dec = get_data("/content/quran_text.txt", 0.8)
s_train, s_test, s_chars, s_enc, s_dec = get_data("/content/sheng_text.txt", 0.8)
n_train, n_test, n_chars, n_enc, n_dec = get_data("/content/news_text.txt", 0.8)

3806991
696677
302833
714004


In [25]:
b_nc, b_vc, b_tn, b_tv = get_ngrams(bbl_train, len_gram)
q_nc, q_vc, q_tn, q_tv = get_ngrams(q_train, len_gram)
s_nc, s_vc, s_tn, s_tv = get_ngrams(s_train, len_gram)
n_nc, n_vc, n_tn, n_tv = get_ngrams(n_train, len_gram)

In [22]:
test_model(q_test, q_nc, q_vc, q_chars, q_enc, len_gram, q_tn, q_tv,split=0.8)

696677
==== Running N-gram model evaluation ====
Current time:  1670573964.0090048
Final loss  [5.458379797870589]
Total time: 140.74948906898499 seconds
==== Finished Running N-gram model evaluation ====


[5.458379797870589]

In [30]:
print("Test: Quran model, Bible dataset: ", test_model(bbl_test, q_nc, q_vc, q_chars, q_enc, len_gram, q_tn, q_tv,split=0.4))

==== Running N-gram model evaluation ====
Current time:  1670574488.9103112
Final loss  [5.473836769121123]
Total time: 383.1017892360687 seconds
==== Finished Running N-gram model evaluation ====
Test: Quran model, Bible dataset:  [5.473836769121123]


In [28]:
print("Test: Quran model, Sheng dataset: ", test_model(s_test, q_nc, q_vc, q_chars, q_enc, len_gram, q_tn, q_tv,split=0.8))

==== Running N-gram model evaluation ====
Current time:  1670574380.6529224
Final loss  [5.524822553024704]
Total time: 65.89717984199524 seconds
==== Finished Running N-gram model evaluation ====
Test: Quran model, Sheng dataset:  [5.524822553024704]


In [31]:
print("Test: Quran model, News dataset: ", test_model(n_test, q_nc, q_vc, q_chars, q_enc, len_gram, q_tn, q_tv,split=0.8))

==== Running N-gram model evaluation ====
Current time:  1670574872.0758226
Final loss  [7.08020121089239]
Total time: 145.99614310264587 seconds
==== Finished Running N-gram model evaluation ====
Test: Quran model, News dataset:  [7.08020121089239]


In [32]:
print("Test: Bible model, Quran dataset: ", test_model(q_test, b_nc, b_vc, bbl_chars, bbl_enc, len_gram, b_tn, b_tv, split=0.8))
print("Test: Bible model, Bible dataset: ", test_model(bbl_test, b_nc, b_vc, bbl_chars, bbl_enc, len_gram, b_tn, b_tv, split=0.4))
print("Test: Bible model, Sheng dataset: ", test_model(s_test, b_nc, b_vc, bbl_chars, bbl_enc, len_gram, b_tn, b_tv, split=0.8))
print("Test: Bible model, News dataset: ", test_model(n_test, b_nc, b_vc, bbl_chars, bbl_enc, len_gram, b_tn, b_tv, split=0.8))

==== Running N-gram model evaluation ====
Current time:  1670575213.364137
Final loss  [5.4727306943982255]
Total time: 166.00264835357666 seconds
==== Finished Running N-gram model evaluation ====
Test: Bible model, Quran dataset:  [5.4727306943982255]
==== Running N-gram model evaluation ====
Current time:  1670575379.3848379
Final loss  [6.267194326601365]
Total time: 434.2763600349426 seconds
==== Finished Running N-gram model evaluation ====
Test: Bible model, Bible dataset:  [6.267194326601365]
==== Running N-gram model evaluation ====
Current time:  1670575813.662682
Final loss  [6.129469875054892]
Total time: 68.11056065559387 seconds
==== Finished Running N-gram model evaluation ====
Test: Bible model, Sheng dataset:  [6.129469875054892]
==== Running N-gram model evaluation ====
Current time:  1670575881.7791495
Final loss  [9.107597064667273]
Total time: 160.80762100219727 seconds
==== Finished Running N-gram model evaluation ====
Test: Bible model, News dataset:  [9.10759706

In [33]:
print("Test: Sheng model, Quran dataset: ", test_model(q_test, s_nc, s_vc, s_chars, s_enc, len_gram, s_tn, s_tv, split=0.8))
print("Test: Sheng model, Bible dataset: ", test_model(bbl_test, s_nc, s_vc, s_chars, s_enc, len_gram, s_tn, s_tv, split=0.4))
print("Test: Sheng model, Sheng dataset: ", test_model(s_test, s_nc, s_vc, s_chars, s_enc, len_gram, s_tn, s_tv, split=0.8))
print("Test: Sheng model, News dataset: ", test_model(n_test, s_nc, s_vc, s_chars, s_enc, len_gram, s_tn, s_tv, split=0.8))

==== Running N-gram model evaluation ====
Current time:  1670576042.618762
Final loss  [5.994444680257829]
Total time: 213.96836066246033 seconds
==== Finished Running N-gram model evaluation ====
Test: Sheng model, Quran dataset:  [5.994444680257829]
==== Running N-gram model evaluation ====
Current time:  1670576256.6039863
Final loss  [5.746255172877149]
Total time: 589.1878063678741 seconds
==== Finished Running N-gram model evaluation ====
Test: Sheng model, Bible dataset:  [5.746255172877149]
==== Running N-gram model evaluation ====
Current time:  1670576845.7931533
Final loss  [5.992289833479651]
Total time: 93.38726687431335 seconds
==== Finished Running N-gram model evaluation ====
Test: Sheng model, Sheng dataset:  [5.992289833479651]
==== Running N-gram model evaluation ====
Current time:  1670576939.1838539
Final loss  [5.910737876156036]
Total time: 222.42512154579163 seconds
==== Finished Running N-gram model evaluation ====
Test: Sheng model, News dataset:  [5.910737876

In [34]:
print("Test: News model, Quran dataset: ", test_model(q_test, n_nc, n_vc, n_chars, n_enc, len_gram, n_tn, n_tv, split=0.8))
print("Test: News model, Bible dataset: ", test_model(bbl_test, n_nc, n_vc, n_chars, n_enc, len_gram, n_tn, n_tv, split=0.4))
print("Test: News model, Sheng dataset: ", test_model(s_test, n_nc, n_vc, n_chars, n_enc, len_gram, n_tn, n_tv, split=0.8))
print("Test: News model, News dataset: ", test_model(n_test, n_nc, n_vc, n_chars, n_enc, len_gram, n_tn, n_tv, split=0.8))

==== Running N-gram model evaluation ====
Current time:  1670577161.6262045
Final loss  [5.7768781903968875]
Total time: 245.37021493911743 seconds
==== Finished Running N-gram model evaluation ====
Test: News model, Quran dataset:  [5.7768781903968875]
==== Running N-gram model evaluation ====
Current time:  1670577407.0144842
Final loss  [5.3939181686100905]
Total time: 666.6594829559326 seconds
==== Finished Running N-gram model evaluation ====
Test: News model, Bible dataset:  [5.3939181686100905]
==== Running N-gram model evaluation ====
Current time:  1670578073.675828
Final loss  [5.499245175526621]
Total time: 108.59894561767578 seconds
==== Finished Running N-gram model evaluation ====
Test: News model, Sheng dataset:  [5.499245175526621]
==== Running N-gram model evaluation ====
Current time:  1670578182.2790904
Final loss  [5.886566542388802]
Total time: 248.3820300102234 seconds
==== Finished Running N-gram model evaluation ====
Test: News model, News dataset:  [5.886566542

### utils

In [8]:
def count_ngrams(text, ngram_dictionary, level = 0):

    # base case
    if len(text) == 1:
      if text[0] in ngram_dictionary:
        ngram_dictionary[text[0]] +=1
      else:
        ngram_dictionary[text[0]] = 1

      return ngram_dictionary
      
    char, text = text[0:1], text[1:]
    if not char in ngram_dictionary.keys():
      ngram_dictionary[char] = {}
    ngram_dictionary[char] = count_ngrams(text, ngram_dictionary[char])

    return ngram_dictionary    

def count_unique(dictionary, counter=0):

  for key in dictionary.keys():
    if isinstance(dictionary[key], dict):
      counter = count_unique(dictionary[key], counter)
    else:
      counter += 1
  
  return counter

def get_ngram_val(text, ngram_dictionary):
  if len(text) == 1:
    if text[0] in ngram_dictionary.keys():
      value = ngram_dictionary[text[0]]
    else:
      value = 0
      return value
  else:
    char, text = text[0:1], text[1:]
    if not char in ngram_dictionary.keys():
      value = 0
    else:
      value = get_ngram_val(text, ngram_dictionary[char])
  
  return value


## Generate sentences

In [ ]:
input_sent = 'maji katika'
sent_len = 100

for i in range(sent_len):
  next_char = suggest_next_char(input_sent, ngram_counts_, vocab, encoder)
  print(input_sent)
  input_sent = input_sent + next_char[0][0]

print(input_sent)



# Keras LSTM

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import numpy as np
import random
import io

In [ ]:
path =r"/content/bible_text_.txt"

with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 3780793
Total chars: 49
Number of sequences: 1260251


<ipython-input-2-a3472af718bd>:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-2-a3472af718bd>:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
del x,y

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import os
from tensorflow.keras.models import save_model, load_model

dir = os.getcwd()

epochs = 20
batch_size = 128

best_model = ""
best_model_loss = inf

for epoch in range(epochs):
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1)
    mdl_loss = history.history["val_loss"]

    if best_model_loss > mdl_loss:
      best_model_loss =  mdl_loss
      path = os.getcwd() + "/model_" +  "loss_" + \
      str(mdl_loss[-1])[0:6].replace(".", "_") + \
      "_model.h5"

      save_model(model, path)

    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

7877/7877 [==============================] - 58s 6ms/step - loss: 1.4641

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "ka? basi sasa, bwana wa majeshi asema hi"
...Generated:  vi, akamwambia mataifa ya mataifa ya mashamba ya mataifa ya mambo hayo ya mataifa, na mataifa ya mataifa yake mwana wa kuu wa mashamba ya mataifa ya mataifa, akamwambia wakati wa mataifa. basi si mataifa huyo mataifa ya mataifa, na mataifa ya mataifa ya kumwondoka kwa maana mataifa, akamwambia, mwana wa adhabuhu ya mataifa ya mambo hayo mwana wa israeli. na mataifa ya mataifa ya mataifa ya makarib

...Diversity: 0.5
...Generating with seed: "ka? basi sasa, bwana wa majeshi asema hi"
...Generated:  vi katika kila mtu aliyewatangaza mambo ya makati ya mashaba, wakamwoka kwa maandiko mwangu, na maandiko ya mataifa ya mashaba, na baba yangu maji, na mataifa, wala hamwamia, na mara yake mwana wa kuoni kwa mambo yake, na kama mtumishi wake yaangu kuu na kukusanyika juu ya kila mtu ni mtumi

<ipython-input-5-ded0137f8096>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  a sitaaminyeshea. ee yeropomo mafu yale naye. tena sawasawa na nguvu za diri, hamareni hilo ule, na kwa tarau aliumbwa vikulali zote, nawe utakuwa na kitakuwa, kwa kirofohthithili kuogopa uje ndugu zako, kuwa arobaini, wepu; wa audheleli wa dogo, jimbi hivi, je! u, wakafika moioni. au hii zake, nianguye na bwana, shedimi na wanane ili mwipupe, kweli, akalawa na kwenu, wasioke haruni kahari ya ngez

7877/7877 [==============================] - 48s 6ms/step - loss: 1.2384

Generating text after epoch: 4
...Diversity: 0.2
...Generating with seed: "u, na sanduku la agano lililofunikwa kwa"
...Generated:   macho ya mataifa ya kutoka katika nchi ya mataifa ya makari ya mataifa na kumwambia, akamwambia, bwana mungu asema hivi, na kumwambia makari ya kusema, mwana wa mungu akamwambia, mwana wa mungu wake waliokuwa wakisema, mwana wa mungu wake walipokuwa wa kuwa makuhani wake, na kumwambia, mwana wa mataifa wake waliokuwa macho yake. na maana wakatia mataifa ya kusema, mwana wa 

KeyboardInterrupt: ignored

In [ ]:
# input = "akawa "
# tokenized_input = [x.lower() for x in input]
# last_gram = tokenized_input[-(len_gram):]
# unique_ngrams = count_unique(ngram_counts)

# char_probs = {}
# for vocab_char in chars:
#   original_gram = test_gram = "".join([last_gram[x] for x in range(len_gram)])

#   test_gram = test_gram[1:]
#   c_vocab = get_ngram_val(test_gram, vocab_counts)

#   test_gram += str(vocab_char)
#   c_ngram = get_ngram_val(test_gram, ngram_counts)
  
#   prob = (c_ngram + 1) /(c_vocab + (1*unique_ngrams))

#   print(test_gram)
#   print(c_ngram + 1 , c_vocab + unique_ngrams, prob)

#   char_probs[vocab_char] = prob    

# top_suggestions = list(char_probs.items())

# totals = sum(map(lambda x: x[1], top_suggestions))
# coeff = 1/totals

# top_suggestions_standardized =  [tuple([x[0], x[1] * coeff]) for x in top_suggestions]